In [24]:
import torch
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
from data_model_RUL import StrainDataSet  # Import the dataset class
from generate_train import generate_train_from_unit_list  # Import the training data generation function
from LSTM_model_RUL import LSTMModel  # Import the LSTM model class

# Set the file path for your dataset
dataset_path = r'D:\DRDO\Remaining-Useful-Life-Prediction-RNN-master\-40 to 160(10 min).xls'  # Update with your actual file path

# Initialize dataset
strain_dataset = StrainDataSet(data_path=dataset_path, num_steps=20, test_ratio=0.2)  # num_steps=20 for sequences of length 20

# Generate the training sequences using the filtered columns
train_X_list, train_Y_list = generate_train_from_unit_list([strain_dataset.train_X], TIMESTEPS=20)

# Flatten the list of arrays into a single numpy array for training
train_X = np.vstack(train_X_list)
train_y = np.vstack(train_Y_list)

# Convert to PyTorch tensors
train_X_tensor = torch.tensor(train_X, dtype=torch.float32)

# Since we are predicting all time steps, use the full train_y without slicing
train_y_tensor = torch.tensor(train_y, dtype=torch.float32)  # Keep all time steps

# Create a DataLoader for batch training
train_data = torch.utils.data.TensorDataset(train_X_tensor, train_y_tensor)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

# Adjust the input size dynamically based on the filtered input shape
input_size = train_X.shape[2]  # Number of features (4 in this case)
print(f"Input size for LSTM: {input_size}")

# Modify the LSTM Model: Output all time steps
class LSTMModelAllSteps(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMModelAllSteps, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)  # Fully connected layer to predict the output

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # Get the output from the LSTM for all time steps
        lstm_out = self.fc(lstm_out)  # Pass through the fully connected layer for each time step
        return lstm_out  # Return predictions for all time steps

# Initialize the modified LSTM model
model = LSTMModelAllSteps(input_size=input_size, hidden_size=64, output_size=1, num_layers=6)

# Set loss function and optimizer
criterion = nn.MSELoss()  # Using Mean Squared Error loss for regression (equivalent strain)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
num_epochs = 200
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()  # Clear the previous gradients
        outputs = model(inputs)  # Forward pass
        
        # Compute the loss across all time steps
        loss = criterion(outputs, targets)
        
        loss.backward()  # Backpropagation
        optimizer.step()  # Update the weights
        
        running_loss += loss.item()
    
    # Print loss for every epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Save the trained model
torch.save(model.state_dict(), 'lstm_equivalent_strain_model_all_steps.pth')

# Example of loading the model later
# model.load_state_dict(torch.load('lstm_equivalent_strain_model_all_steps.pth'))

# Fix the test data processing:
# Ensure that test_X has the same number of features as train_X (4 features)
test_X_list, test_Y_list = generate_train_from_unit_list([strain_dataset.test_X], TIMESTEPS=20)
test_X = np.vstack(test_X_list)
test_y = np.vstack(test_Y_list)

# Ensure that test_X has only the required 4 features (consistent with training)
test_X = test_X[:, :, :input_size]  # Slice to keep the first 4 features (based on input_size)

print(f"test_X shape after selection: {test_X.shape}")  # Should now match train_X shape

# Convert to PyTorch tensors for testing
test_X_tensor = torch.tensor(test_X, dtype=torch.float32)
test_y_tensor = torch.tensor(test_y, dtype=torch.float32)  # Keep all time steps for testing

# Forward pass on test data
with torch.no_grad():  # No need to track gradients during evaluation
    test_outputs = model(test_X_tensor)
    test_loss = criterion(test_outputs, test_y_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

# Convert predictions and actual values back to numpy for plotting
predicted = test_outputs.numpy().flatten()
actual = test_y_tensor.numpy().flatten()

# For example, plot only the first 1000 samples (you can adjust this number)
num_samples_to_plot = 50
plt.figure(figsize=(14, 6))  # Increase the width (14) while keeping the height (6)
plt.plot(actual[:num_samples_to_plot], label='Actual Equivalent Strain', color='blue')
plt.plot(predicted[:num_samples_to_plot], label='Predicted Equivalent Strain', color='red', linestyle='--')
plt.title('Actual vs Predicted Equivalent Strain (First 1000 Samples)')
plt.xlabel('Sample')
plt.ylabel('Equivalent Strain')
plt.legend()
plt.grid(True)
plt.show()


num_steps: 20
Input size for LSTM: 39
Epoch [1/200], Loss: 0.2044
Epoch [2/200], Loss: 0.1176
Epoch [3/200], Loss: 0.1097
Epoch [4/200], Loss: 0.1062
Epoch [5/200], Loss: 0.1036
Epoch [6/200], Loss: 0.1016
Epoch [7/200], Loss: 0.0998
Epoch [8/200], Loss: 0.0951
Epoch [9/200], Loss: 0.0541
Epoch [10/200], Loss: 0.0308
Epoch [11/200], Loss: 0.0255
Epoch [12/200], Loss: 0.0229
Epoch [13/200], Loss: 0.0218
Epoch [14/200], Loss: 0.0212
Epoch [15/200], Loss: 0.0207
Epoch [16/200], Loss: 0.0203
Epoch [17/200], Loss: 0.0204
Epoch [18/200], Loss: 0.0197
Epoch [19/200], Loss: 0.0194
Epoch [20/200], Loss: 0.0192
Epoch [21/200], Loss: 0.0187
Epoch [22/200], Loss: 0.0182
Epoch [23/200], Loss: 0.0179
Epoch [24/200], Loss: 0.0175
Epoch [25/200], Loss: 0.0171
Epoch [26/200], Loss: 0.0167
Epoch [27/200], Loss: 0.0158
Epoch [28/200], Loss: 0.0153
Epoch [29/200], Loss: 0.0146
Epoch [30/200], Loss: 0.0138
Epoch [31/200], Loss: 0.0131
Epoch [32/200], Loss: 0.0120
Epoch [33/200], Loss: 0.0111
Epoch [34/200]

In [6]:
import pandas as pd

# Load the Excel file
dataset_path = r"D:\DRDO\Remaining-Useful-Life-Prediction-RNN-master\-40 to 160(10 min).xls"
data = pd.read_excel(dataset_path)

# Print the actual column names
print(data.columns)


Index(['Strain enegry', 'Total deformation', 'Equivalent  strain',
       'max Shear strain', 'Number of cycles to failure'],
      dtype='object')


In [18]:
dataset_path = "D:\DRDO\Remaining-Useful-Life-Prediction-RNN-master\-40 to 160(10 min).xls"  # Update with your actual file path


In [19]:
import pandas as pd

# Load the dataset
data = pd.read_excel(dataset_path)

# Print the column names
print(data.columns)


Index(['Strain energy', 'Total deformation', 'Equivalent strain',
       'max Shear strain', 'Number of cycles to failure'],
      dtype='object')
